In [1]:
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding
import datasets
from datasets import load_metric
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import sys
import json

In [2]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def save2Json(metrics, filename):
    json_object = json.dumps(metrics, indent=4)
    with open(filename, "w") as outfile:
        outfile.write(json_object)


def run_benchmark_single(model_name, tokenizer_checkpoint, model_checkpoint, SPARSE_PERCENT):
    raw_datasets = datasets.load_dataset('glue', 'mrpc')
    
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
    def tokenize_function(sample):
        return tokenizer(sample['sentence1'], sample['sentence2'], truncation=True)
    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

    training_args = TrainingArguments(
        num_train_epochs = 1,
        output_dir=f'output/glue-{model_name}_{SPARSE_PERCENT}/',
    )

    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator, 
        tokenizer=tokenizer,
        # compute_metrics=compute_metrics
    )
    print(f"Start training {model_checkpoint}...")

    trains = trainer.train()
    # predictions = trainer.predict(tokenized_datasets['validation'])
    save2Json(trains.metrics, filename=f'output/glue-{model_name}_{SPARSE_PERCENT}.json')


In [5]:
run_benchmark_single(
    model_name="gpt2", 
    tokenizer_checkpoint='./models/gpt2-glue-tokenizer/', 
    model_checkpoint="./models/gpt2-glue_0.1/", 
    SPARSE_PERCENT=10
)

Found cached dataset glue (/Users/shuyuzhou/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Loading cached processed dataset at /Users/shuyuzhou/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-96e198acbca671c6.arrow
Loading cached processed dataset at /Users/shuyuzhou/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e15b3c8aa5ebf795.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

loading configuration file ./models/gpt2-glue_0.1/config.json
Model config GPT2Config {
  "_name_or_path": "./models/gpt2-glue_0.1/",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2ForSequenceClassification"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.2

Start training ./models/gpt2-glue_0.1/...


  0%|          | 0/459 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 